# End-to-end Multi-class Dog Breed Classification

This notebook builds an end-to-end multi-class image classifier using TensorFlow 2.0 and TensorFlow Hub.

## 1.Problem
Identifying the breed of a dog given an image of a dog.

Given a photo of a dog, we want to know breed of the dog.

## 2. Data

The data we're using is from Kaggle's dog breeed identification competition.

https://www.kaggle.com/c/dog-breed-identification/data

## 3. Evaluation

The evaluation is a file with prediction probabilites for each dog breed for each test image.

https://www.kaggle.com/c/dog-breed-identification/overview/evaluation

## 4. Features

Some information about the data:
* We're dealing with images (unstructured data), so we are using deep learning/transfer learning.

* There are 120 breeds of dogs (this means there are 120 different classes).

* There are around 10000+ images in the training set (these images have labels).

* There are around 10000+ images in the test set( labels of these images are to be predicted).


###  Get our workspace ready

In [7]:
# Import Tensorflow into Colab

import tensorflow as tf

print("TF version",tf.__version__)

TF version 2.3.0
